In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

!pip install optuna 

# category variable
category = ['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9']

# continuous variable
continuous = ['cont0', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 
              'cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13']

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
train_dataset = pd.read_csv('../input/tabular-playground-series-feb-2021/train.csv')
test_data = pd.read_csv('../input/tabular-playground-series-feb-2021/test.csv')
dataset = pd.concat([train_dataset, test_data])
#print(dataset.info())

# あきらかな外れ値は削除
# [166042]はあきらかな外れ値
target_mean = dataset.describe().loc['mean', 'target']
target_std = dataset.describe().loc['std', 'target']
up = target_mean+4*target_std
low=target_mean-4*target_std

outlier = dataset.loc[dataset['target'] < low, 'id'].values
#outlier=[166042]
for x in outlier:
    dataset = dataset.loc[dataset['id'] != x, :] 

# idとtargetは避難させておく
id = dataset['id']
target = dataset['target']
# 相関が高いものだけを使用。この手法は有効だったと思うが、Catboostでのスコアは一定以上伸びなかった。
 
    
## ***** Feature Engineering - Create new columns ***** ##
#for x in range(0,10):
#    dataset[format(x, '03')] = dataset[('cont' + str(x))] * dataset[('cat' + str(x))]
## **************************************************** ##    


## ***** Encoding ***** ##
from sklearn.preprocessing import LabelEncoder, RobustScaler, StandardScaler
scaler = RobustScaler()
encoder = LabelEncoder()

cat_index = []
for col in dataset.columns:
#    # カテゴリ変数はエンコーディングを適用
    if dataset[col].dtype == object:
        cat_index.append(dataset.columns.get_loc(col))
        dataset[col] = encoder.fit_transform(dataset[col])

#dataset[continuous] = scaler.fit_transform(dataset.loc[:,continuous].values)

# targetのあるなしでtrainとtestを分割
train = dataset.loc[dataset['target'].notnull(), :]
test  = dataset.loc[dataset['target'].isnull(), :]

X = train.drop(columns=['id', 'target'])
y = train['target']
X_prediction = test.drop(columns=['id', 'target'])
prediction_id = test.loc[:,'id']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

dataset

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,0,1,0,0,1,3,0,4,2,...,0.881122,0.421650,0.741413,0.895799,0.802461,0.724417,0.701915,0.877618,0.719903,6.994023
1,2,1,0,0,0,1,1,0,4,0,...,0.440011,0.346230,0.278495,0.593413,0.546056,0.613252,0.741289,0.326679,0.808464,8.071256
2,3,0,0,0,2,1,3,0,1,2,...,0.914155,0.369602,0.832564,0.865620,0.825251,0.264104,0.695561,0.869133,0.828352,5.760456
3,4,0,0,0,2,1,3,0,4,6,...,0.934138,0.578930,0.407313,0.868099,0.794402,0.494269,0.698125,0.809799,0.614766,7.806457
4,6,0,1,0,0,1,1,0,4,2,...,0.382600,0.705940,0.325193,0.440967,0.462146,0.724447,0.683073,0.343457,0.297743,6.868974
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,499987,0,0,0,2,1,3,0,4,6,...,0.963678,0.240482,0.686462,0.915165,0.848878,0.459598,0.590327,0.864873,0.425258,NaN
199996,499990,0,0,0,2,1,3,0,4,4,...,0.232072,0.363421,0.694092,0.137002,0.319465,0.364527,0.388908,0.664357,0.224215,NaN
199997,499991,0,0,0,2,1,3,0,4,2,...,0.432927,0.811876,0.328398,0.496017,0.538779,0.466338,0.643869,0.749590,0.457702,NaN
199998,499994,0,1,0,0,1,3,0,4,2,...,0.837712,0.680886,0.534439,0.501588,0.809053,0.631704,0.766426,0.937139,0.796304,NaN


In [14]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import statistics as stat
import lightgbm as lgb

def kfold_processing(models, X, y):
    kf = KFold(n_splits=5,random_state=48,shuffle=True)
    rmse=[]  # list contains rmse for each fold
    n=0
    preds=0
    
    for model in models:
        # kf.splitはindexを返すことに注意。データ自体じゃないよ!
        for train, test in kf.split(X, y):
            x_tr = X.iloc[train, :]
            x_te = X.iloc[test,  :]
            y_tr = y.iloc[train]
            y_te = y.iloc[test]
            model.fit(x_tr,y_tr,eval_set=[(x_te,y_te)],early_stopping_rounds=100,verbose=False)
            rmse_metric = mean_squared_error(y_te, model.predict(x_te), squared=False)
            rmse.append(rmse_metric)
            preds+=model.predict(X_prediction)
            print(n+1,rmse[n])
            n+=1

        print(f'RMSE mean = {stat.mean(rmse)}')
        print(f'RMSE = {rmse}')
    return preds

In [13]:
# sklearn.ensemble.StackingRegressor — scikit-learn 0.24.1 documentation 
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingRegressor.html

import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import StackingRegressor
best_lgb_params = {'objective': 'regression', 'num_leaves': 87, 'max_depth': 9, 'learning_rate': 0.009582240516938432, 'n_estimators': 2553, 
                   'reg_alpha': 0.037580598353005736, 'reg_lambda': 0.02686007922451687, 'colsample_bytree': 0.5534542046480458}

best_xgb_params = {'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'tree_method': 'gpu_hist','eta': 0.6967322281466614, 
                   'gamma': 0.0036942646535044962, 'max_depth': 6, 'sub_sample': 0.553569633282626, 
                   'colsample_bytree': 0.554663669927102, 'lambda': 0.001701369247469206, 'alpha': 0.009630419498745284, 
                   'learning_rate': 0.03446986865236482, 'n_estimators': 603}      

lgb_model = lgb.LGBMRegressor(**best_lgb_params)
xgb_model = xgb.XGBRegressor(**best_xgb_params)
kf = KFold(n_splits=5,random_state=48,shuffle=True)

models = [lgb_model]

y_pred = kfold_processing(models=models, X=X, y=y)

# StackingRegressorのトライ
'''
estimators = [
    ('lgb', lgb_model),
    ('xgb_model', xgb_model)
]

streg = StackingRegressor(estimators=estimators, cv=kf)
streg.fit(X,y)
'''

1 0.8445690179759722
2 0.8423412780092387
3 0.8423321367289558
4 0.8422793257727865
5 0.8440768228325857
RMSE mean = 0.8431197162639078
RMSE = [0.8445690179759722, 0.8423412780092387, 0.8423321367289558, 0.8422793257727865, 0.8440768228325857]


"\nestimators = [\n    ('lgb', lgb_model),\n    ('xgb_model', xgb_model)\n]\n\nstreg = StackingRegressor(estimators=estimators, cv=kf)\nstreg.fit(X,y)\n"

### "num_iteration=model.best_iteration_"があるとき
> 
1 0.8445690179759722  
2 0.8423412780092387  
3 0.8423321367289558  
4 0.8422793257727865  
5 0.8440768228325857  
RMSE mean = 0.8431197162639078  
RMSE = [0.8445690179759722, 0.8423412780092387, 0.8423321367289558, 0.8422793257727865, 0.8440768228325857]

### "num_iteration=model.best_iteration_"がないとき
>
1 0.8445690179759722  
2 0.8423412780092387  
3 0.8423321367289558  
4 0.8422793257727865  
5 0.8440768228325857  
RMSE mean = 0.8431197162639078  
RMSE = [0.8445690179759722, 0.8423412780092387, 0.8423321367289558, 0.8422793257727865, 0.8440768228325857]


### Optuna Tuning
[lightgbm.LGBMRegressor — LightGBM 3.1.1.99 documentation](https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMRegressor.html)  
[XGBoostパラメータのまとめとランダムサーチ実装 - Qiita](https://qiita.com/FJyusk56/items/0649f4362587261bd57a)  
[XGBoost Parameters — xgboost 1.4.0-SNAPSHOT documentation](https://xgboost.readthedocs.io/en/latest/parameter.html)  
[CatBoostRegressor - CatBoost. Documentation](https://catboost.ai/docs/concepts/python-reference_catboostregressor.html)

In [7]:
# Lightgbm Optuna

import lightgbm as lgb
from sklearn.metrics import r2_score, mean_squared_error
import optuna

trains = lgb.Dataset(X_train, y_train)
tests = lgb.Dataset(X_test, y_test)

def objective(trial):
    params = {
        'objective': 'regression',
        'num_leaves': trial.suggest_int('num_leaves', 5, 200),
        'max_depth': trial.suggest_int('max_depth', 4, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 0.001, 0.1),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 0.001, 0.1),
        'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.5, 1),
        'random_state': 42
    }  
    
    
    lgb_model = lgb.LGBMRegressor(**params)
    lgb_model.fit(X=X_train, y=y_train, eval_set = [(X_test, y_test)], eval_metric='rmse',  early_stopping_rounds=10, verbose=False)
    y_pred = lgb_model.predict(X_test, num_iteration=lgb_model.best_iteration_)
    
    return mean_squared_error(y_test, y_pred)

study = optuna.create_study()
study.optimize(objective, n_trials=50, n_jobs=-1)
lgb_best = study.best_params
print(lgb_best)

[I 2021-03-01 02:04:13,257] A new study created in memory with name: no-name-d32588ad-ed2c-4d59-a525-b2020472a287
[I 2021-03-01 02:05:27,558] Trial 0 finished with value: 0.713292361998873 and parameters: {'num_leaves': 85, 'max_depth': 9, 'learning_rate': 0.004856916551979603, 'n_estimators': 2834, 'reg_alpha': 0.0021415442776480803, 'reg_lambda': 0.005469639847921993, 'colsample_bytree': 0.91094607014517}. Best is trial 0 with value: 0.713292361998873.


{'num_leaves': 85, 'max_depth': 9, 'learning_rate': 0.004856916551979603, 'n_estimators': 2834, 'reg_alpha': 0.0021415442776480803, 'reg_lambda': 0.005469639847921993, 'colsample_bytree': 0.91094607014517}


In [ ]:
# XGBoost Optuna

import xgboost as xgb
from sklearn.metrics import r2_score, mean_squared_error
import optuna

# XGBoost Parameters — xgboost 1.4.0-SNAPSHOT documentation https://xgboost.readthedocs.io/en/latest/parameter.html

def objective(trial):
    params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'tree_method': 'gpu_hist',
        'eta': trial.suggest_loguniform('eta', 0.1, 1.0),
        'gamma': trial.suggest_loguniform('gamma', 0.001, 5.),
        'max_depth': trial.suggest_int('max_depth', 4, 10),
        'sub_sample': trial.suggest_loguniform('sub_sample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.5, 1.0),
        'lambda': trial.suggest_loguniform('lambda', 0.001, 0.01),
        'alpha': trial.suggest_loguniform('alpha', 0.001, 0.01),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.0001, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000)
    }  
    
    xgb_model = xgb.XGBRegressor(**params)
    xgb_model.fit(X_train, y_train, eval_set = [(X_test, y_test)],  early_stopping_rounds=10)
    y_pred = xgb_model.predict(X_test)
    
    return np.sqrt(mean_squared_error(y_test, y_pred))

study = optuna.create_study()
study.optimize(objective, n_trials=50, n_jobs=-1)
xgb_best = study.best_params
print(xgb_best)

In [ ]:
## CatBoost Optuna

from catboost import CatBoostRegressor
from catboost import Pool
from sklearn.metrics import r2_score, mean_squared_error
import optuna

cat_train = Pool(X_train, y_train, cat_features=category)
cat_test = Pool(X_test, y_test, cat_features=category)

def objective(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 100, 3000),
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        'l2_leaf_reg': trial.suggest_int('l2_leaf', 1,10),
        'eval_metric': 'RMSE',
        'random_strength': trial.suggest_int('random_strength', 0, 100),
        'bagging_temperature': trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
        'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
        'od_wait': trial.suggest_int('od_wait', 10, 50),
        'use_best_model': True
    }  
    
    model = CatBoostRegressor(**params)
    model.fit(cat_train, eval_set=cat_test, early_stopping_rounds=10, verbose=0)
    y_pred = model.predict(cat_test)
    
    return np.sqrt(mean_squared_error(y_test, y_pred))

study = optuna.create_study()
study.optimize(objective, n_trials=100, n_jobs=-1)
cat_best = study.best_params
print(cat_best)
print(f'Best Value: {study.best_value}')

In [ ]:
## prediction後に結果をファイルへ出力

#y_pred=regressor.predict(X_prediction)
print(y_pred/10)
output = pd.DataFrame({'id': prediction_id, 'target': y_pred/10})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

In [ ]:
## (参考) Lightgbm Optuna

import optuna.integration.lightgbm as lgb

trains = lgb.Dataset(X_train, y_train)
tests = lgb.Dataset(X_test, y_test)

params = {'objective': 'mean_squared_error',
         'metric': 'rmse'}

lgb_model = lgb.train(params, trains, valid_sets=tests, early_stopping_rounds=10)
best_params = lgb_model.params


import lightgbm as lgb

lgb_model = lgb.train(best_params, trains, valid_sets=tests)
y_pred = lgb_model.predict(X_test)

from sklearn.metrics import mean_squared_error
print(mean_squared_error(y_test, y_pred))

In [ ]:
## (参考) TensorFlow Regression
import tensorflow as tf
from tensorflow import keras

def rmse(y_true, y_pred):
    return tf.sqrt(tf.losses.mean_squared_error(y_true, y_pred))

### ***********************************************************###
model = tf.keras.Sequential()

#model.add(tf.keras.layers.Dense(1024, activation='relu', input_shape=(X.shape[1],)))
#model.add(tf.keras.layers.LeakyReLU())
#model.add(tf.keras.layers.BatchNormalization())
#model.add(tf.keras.layers.Dropout(0.5))

#model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dense(64, kernel_regularizer=keras.regularizers.l2(0.001), activation='relu',input_shape=(X.shape[1],)))
model.add(tf.keras.layers.LeakyReLU())
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.4))

#model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dense(64, kernel_regularizer=keras.regularizers.l2(0.001), activation='relu'))
model.add(tf.keras.layers.LeakyReLU())
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.4))

#model.add(tf.keras.layers.Dense(units=256, activation='relu'))
model.add(tf.keras.layers.Dense(32, kernel_regularizer=keras.regularizers.l2(0.001), activation='relu'))
model.add(tf.keras.layers.LeakyReLU())
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.3))

#model.add(tf.keras.layers.Dense(units=256, activation='relu'))
model.add(tf.keras.layers.Dense(32, kernel_regularizer=keras.regularizers.l2(0.001), activation='relu'))
model.add(tf.keras.layers.LeakyReLU())
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.3))

#model.add(tf.keras.layers.Dense(units=128, activation='relu'))
model.add(tf.keras.layers.Dense(16, kernel_regularizer=keras.regularizers.l2(0.001), activation='relu'))
model.add(tf.keras.layers.LeakyReLU())

model.add(tf.keras.layers.Dense(units=1, activation='linear'))
### ***********************************************************###

optimizer = tf.keras.optimizers.Adam(lr=0.005, decay=5e-4)
model.compile(optimizer = optimizer, loss = 'mae', metrics = ['mse', 'mae'])

#checkpoint_name = 'Model/{epoch:03d}-{val_loss:.5f}.hdf5'
checkpoint_name = 'DNN_BestModel.hdf5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
callback_list = [checkpoint]

history = model.fit(X_train, y_train, validation_split=0.2, epochs = 500, batch_size = 1024,
                    validation_data=(X_test, y_test), callbacks=callback_list)
y_pred = model.predict(X_test).reshape(-1)
RMSE = rmse(y_test, y_pred)
print(f'RMSE = {RMSE}')

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(111)
ax.set_ylim(0.6, 0.75)

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['Train', 'Test'], loc='upper right')
plt.savefig(fname='1024 neurons enable LearningRate.png')